# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,2.88,94,100,3.77,IS,1718762004
1,1,yulinshi,38.2906,109.7494,28.73,39,12,5.51,CN,1718761740
2,2,stanley,54.8680,-1.6985,9.01,94,75,1.54,GB,1718762005
3,3,flying fish cove,-10.4217,105.6791,26.82,78,75,7.20,CX,1718762005
4,4,mugia,43.1041,-9.2179,12.31,88,86,1.35,ES,1718762005


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True, 
    tiles = 'OSM', 
    frame_width = 980,
    size = 'Humidity', 
    color = 'City')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
visit_cities = city_data_df.loc[(city_data_df['Max Temp'] >= 21) & 
                                (city_data_df['Max Temp'] <= 27) & 
                                (city_data_df['Wind Speed'] <= 4.5) & 
                                (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
visit_cities = visit_cities.dropna()

# Display sample data
visit_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,kill devil hills,36.0307,-75.6760,24.43,78,0,1.54,US,1718762011
37,37,jeju city,33.5097,126.5219,26.96,57,0,1.54,KR,1718762016
96,96,nova vicosa,-17.8919,-39.3719,22.43,82,0,2.52,BR,1718762032
102,102,noumea,-22.2763,166.4572,25.00,99,0,4.29,NC,1718761810
112,112,arenapolis,-14.4503,-56.8461,22.22,43,0,1.88,BR,1718762037
113,113,concepcion del oro,24.6333,-101.4167,21.51,28,0,4.33,MX,1718762037
137,137,kone,-21.0595,164.8658,25.63,39,0,3.45,NC,1718762043
175,175,remire-montjoly,4.9167,-52.2667,25.02,100,0,1.54,GF,1718762054
190,190,dogonbadan,30.3586,50.7981,26.05,31,0,0.00,IR,1718762058
203,203,emerald,-23.5333,148.1667,21.30,20,0,0.71,AU,1718761973


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame().assign(City=visit_cities['City'], Country=visit_cities['Country'], Lat=visit_cities['Lat'], Lng=visit_cities['Lng'], Humidity=visit_cities['Humidity'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
21,kill devil hills,US,36.0307,-75.6760,78,
37,jeju city,KR,33.5097,126.5219,57,
96,nova vicosa,BR,-17.8919,-39.3719,82,
102,noumea,NC,-22.2763,166.4572,99,
112,arenapolis,BR,-14.4503,-56.8461,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'filter':'',
    'bias':''
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:'+str(lng)+','+str(lat)+','+str(radius)
    params["bias"] = 'proximity:'+str(lng)+','+str(lat)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = base_url+'?categories=accommodation.hotel&filter='+params['filter']+'&bias='+params['bias']+'&limit=1&apiKey='+geoapify_key
    
    # Convert the API response to JSON format
    name_address = requests.get(name_address).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kill devil hills - nearest hotel: Mariner Days Inn & Suites
jeju city - nearest hotel: R&T 호텔
nova vicosa - nearest hotel: Pousada Pontal da Barra
noumea - nearest hotel: Hotel Gondwana
arenapolis - nearest hotel: No hotel found
concepcion del oro - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
remire-montjoly - nearest hotel: Complexe Belova
dogonbadan - nearest hotel: هتل پاسارگاد
emerald - nearest hotel: Emerald Central Hotel
mytilene - nearest hotel: ΛΑ ΜΙΡΑΖ
santana - nearest hotel: Hotel Teodoro
mount isa - nearest hotel: Ibis Styles
al bawiti - nearest hotel: Old Oasis Hotel
sabzevar - nearest hotel: مامورسرا مخابرات سبزوار
paphos - nearest hotel: Mayfair Gardens Hotel
xilitla - nearest hotel: Cielito Lindo
scordia - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
filiatra - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
21,kill devil hills,US,36.0307,-75.6760,78,Mariner Days Inn & Suites
37,jeju city,KR,33.5097,126.5219,57,R&T 호텔
96,nova vicosa,BR,-17.8919,-39.3719,82,Pousada Pontal da Barra
102,noumea,NC,-22.2763,166.4572,99,Hotel Gondwana
112,arenapolis,BR,-14.4503,-56.8461,43,No hotel found
113,concepcion del oro,MX,24.6333,-101.4167,28,No hotel found
137,kone,NC,-21.0595,164.8658,39,Pacifik Appartels
175,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
190,dogonbadan,IR,30.3586,50.7981,31,هتل پاسارگاد
203,emerald,AU,-23.5333,148.1667,20,Emerald Central Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    color = 'City',
    frame_width = 980,
    frame_height = 500,
    hover_cols = ['Humidity','Hotel Name','Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)